In [40]:
from thermo import *
from thermo.identifiers import ChemicalMetadataDB
from numpy.testing import assert_allclose
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.rdMolDescriptors import CalcMolFormula

a = ChemicalMetadataDB(elements=False, main_db=('last_results.txt'), user_dbs=[])

a

In [41]:
data = {}
with open('Original metadata.csv') as f:
    f.readline()
    for line in f.readlines():
        name, name2, CAS, formula, charge, MW = line.split('\t')
        data[CAS] = {'Name': name, 'Name2': name2, 'formula': formula, 'charge':int(charge), 'MW': float(MW)}

In [4]:
# good_syns = {CAS:{'synonyms': [d['Name2']]} for CAS, d in data.items()}

# for CAS, d in data.items():
#     formula = d['formula'].split('+')[0]
#     charge = d['charge']
#     formulas = [formula+'+'*charge, formula+'+'+str(charge),
#                formula+'('+'+'+ str(charge) + ')', formula+'('+ str(charge)+'+' + ')']
#     good_syns[CAS]['synonyms'].extend(formulas)

    
# good_syns['14464-47-2']['synonyms'].append('deuterium(1+)')

# import json
# f = open('Good synoynms by CAS.json', 'w')
# json.dump(good_syns, f, indent=2, separators=(',', ': '), sort_keys=True)

In [42]:
from collections import Counter
ns = []
for i in a.CAS_index.values():
    ns.extend(list(set(i.all_names)))
# Counter(ns).most_common(20)
# No dup names :)
# len(ns)
len(a.CAS_index), len(a.pubchem_index)

(124, 106)

In [45]:
# None of the charges are wrong?
for CAS, d in data.items():
    chem = a.search_CAS(CAS)
    if not chem:
        continue
        print('NOTINDB', CAS)
        continue
    mol = Chem.MolFromSmiles(chem.smiles)
    if mol is None:
        continue
        print('CANTREADMOL', CAS)
        continue
#     print(Chem.MolToSmiles(mol))
    charge = Chem.GetFormalCharge(mol)
    try:
        assert charge == d['charge']
#         print('PASS', charge, d['charge'])
    except:
        print('F:', charge, d['charge'], CAS)

('F:', 3, -1, '19469-81-9')
('F:', 0, -1, '11062-77-4')
('F:', -8, -4, '17923-49-8')
('F:', 3, -1, '16971-29-2')


In [4]:
# Chem.GetFormalCharge(Chem.MolFromSmiles('[SbH6+3]'))

In [6]:
# mol = Chem.MolFromMolFile('mol/14695-95-5.mol')
# mol = Chem.MolFromMolFile('/tmp/399316.mol')

# # # mol = Chem.MolFromSmiles('[Sb+3]')
# # # When read, 1 atom
# # Chem.MolToSmiles(mol, allHsExplicit=True)
# # mol.GetNumAtoms()
# mw = Descriptors.MolWt(mol)
# formula = CalcMolFormula(mol)
# mw, formula

In [47]:
# Most of the MW ones fail due to having added extra hydrogens???? OR MW?
for CAS, d in data.items():
    chem = a.search_CAS(CAS)
    if not chem:
        continue
    try:
        assert_allclose(chem.MW, d['MW'], atol=0.3)
    except:
        print('F:', CAS, chem.MW, d['MW'],  chem)

# Plenty of outstanding work here and with the charges.


('F:', '20561-39-1', 210.0, 211.0, <ChemicalMetadata, name=Astatine ion (At1-), formula=At-, smiles=[At-], MW=210>)
('F:', '16518-47-1', 140.934, 116.95, <ChemicalMetadata, name=Arsenate (AsO43-), dihydrogen (8CI,9CI), formula=H2AsO4-, smiles=O=[As]([O-])(O)O, MW=140.934>)
('F:', '14493-01-7', 514.694, 515.71, <ChemicalMetadata, name=Platinate(2-), tetrabromo-, (SP-4-1)-, formula=Br4Pt-2, smiles=[Br-].[Br-].[Br-].[Br-].[Pt+2], MW=514.694>)
('F:', '19469-81-9', 26.982, 78.84, <ChemicalMetadata, name=Aluminate(1-), tetrahydro-, (T-4)- (9CI), formula=Al+3, smiles=[Al+3], MW=26.982>)
('F:', '14897-04-2', 215.041, 216.11, <ChemicalMetadata, name=Cobaltate(3-), hexakis(cyano-κC)-, (OC-6-11)-, formula=C6CoN6-3, smiles=[C-]#N.[C-]#N.[C-]#N.[C-]#N.[C-]#N.[C-]#N.[Co+3], MW=215.041>)
('F:', '34786-97-5', 114.938, 139.94, <ChemicalMetadata, name=Vanadate (VO43-), dihydrogen (8CI), formula=O4V-, smiles=[O-].[O-].[O].[O].[V+], MW=114.938>)
('F:', '14333-20-1', 161.996, 163.01, <ChemicalMetadata, nam